# GPU

In [1]:
!nvidia-smi

Sat Jul  1 23:35:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:09:00.0 Off |                  N/A |
|  0%   36C    P8     5W / 300W |      1MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:0A:00.0 Off |                  N/A |
|  0%   

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #aqui tem q escolher uma das gpus, veja a que esta desocupada (comando: nvidia-smi)
tf_device='/gpu:0'

# Imports

In [3]:
import cv2
import glob
import joblib
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from skimage.feature import greycomatrix, greycoprops
from matplotlib import pyplot as plt
%matplotlib inline

# Dataset

In [4]:
# Carregando as imagens da base
dataset_negativo = sorted(glob.glob('dataset/Negativo/*.jpg'))
dataset_positivo = sorted(glob.glob('dataset/Positivo/*.jpg'))
dataset_outras = sorted(glob.glob('dataset/outras/*.JPEG'))

dataset = dataset_negativo+dataset_positivo+dataset_outras

In [5]:
print("Dataset negativo {}; Dataset positivo {}; Dataset outras {}".format(len(dataset_negativo), len(dataset_positivo), len(dataset_outras)))

Dataset negativo 72; Dataset positivo 78; Dataset outras 150


In [6]:
labels = np.concatenate((np.ones(len(dataset_negativo)+len(dataset_positivo)), np.zeros(len(dataset_outras))))
labels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Extração das características

In [7]:
def calculate_glcm_features(image):
    
    # Converta a imagem para escala de cinza
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calcule a matriz GLCM
    glcm = greycomatrix(gray_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)

    # Calcule as propriedades da matriz GLCM
    contrast = greycoprops(glcm, 'contrast')
    dissimilarity = greycoprops(glcm, 'dissimilarity')
    homogeneity = greycoprops(glcm, 'homogeneity')
    energy = greycoprops(glcm, 'energy')
    correlation = greycoprops(glcm, 'correlation')
    
    return contrast, dissimilarity, homogeneity, energy, correlation

In [8]:
features = np.zeros((len(dataset), 5)) 

# Carregue as imagens de treinamento e suas respectivas classes (positivo/negativo)
def load_images_and_labels():
    
    # Percorra seu conjunto de treinamento e carregue as imagens e rótulos
    for id_img, image_path in enumerate(dataset):
        image = cv2.imread(image_path)
        features[id_img,:] = calculate_glcm_features(image)
        
    return np.array(features), np.array(labels)

In [9]:
# Carregue as imagens e rótulos de treinamento
train_images, train_labels = load_images_and_labels()

In [10]:
train_images.shape, train_labels.shape

((300, 5), (300,))

In [11]:
# Divida o conjunto de treinamento em conjuntos de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [12]:
# Crie um classificador Random Forest
rf_classifier = RandomForestClassifier(n_estimators=1000, n_jobs = -1, random_state=42)

In [13]:
# Treine o classificador
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=42)

In [14]:
# Avalie o desempenho do classificador no conjunto de validação
accuracy = rf_classifier.score(X_val, y_val)
print("Accuracy:", accuracy)

Accuracy: 1.0


# Salvar classificador

In [15]:
# Salva o classificador treinado em um arquivo
joblib.dump(rf_classifier, 'modelos_webapp/random_forest_model.joblib')

['modelos_webapp/random_forest_model.joblib']

# Predição de uma imagem

In [16]:
# Carrega o modelo salvo a partir do arquivo
loaded_model = joblib.load('modelos_webapp/random_forest_model.joblib')

In [17]:
# Carregue a imagem que você deseja fazer a predição
image_path = 'dataset/Captura de tela de 2022-06-17 14-16-22.png'
image = cv2.imread(image_path)

In [18]:
# Extraia as características GLCM da imagem
features = calculate_glcm_features(image)
features

(array([[317.74139598]]),
 array([[3.54668326]]),
 array([[0.77940005]]),
 array([[0.42797303]]),
 array([[0.98277455]]))

In [19]:
# Redimensione as características para que tenham a mesma forma usada durante o treinamento
features = np.reshape(features, (1, -1))
features

array([[317.74139598,   3.54668326,   0.77940005,   0.42797303,
          0.98277455]])

In [20]:
# Faça a predição usando o modelo treinado
prediction = loaded_model.predict(features)
prediction

array([0.])

In [21]:
# Definir o limiar
limiar = 0.5

# Classificar com base no limiar
if prediction >= limiar:
    prediction = "Positivo"
else:
    prediction = "Negativo"
    
print(prediction)

Negativo
